In [1]:
import pandas as pd
import numpy as np
import gc
import os, shutil
import json
import re
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from sklearn.neighbors import NearestNeighbors
import open3d as o3d
import copy
import transforms3d

In [2]:
%matplotlib notebook

In [3]:
BASE_PATH = "/home/sabyasachi/Projects/ati/data/data/datasets/Carla/lidarParam1/"
LIDAR_RANGE = 25
USE_EXTENT = True


# STATIC_FOLDER = "static"
DYNAMIC_FOLDER = "dynamic"
NONPLAYERS_FOLDER = "_nonplayers"
PCD_FOLDER = "_out"
SEGMENT_FOLDER = "_segment"
STATIC_ONLY_FOLDER = "_static_only"
ANNOT_FILE = "annotation.csv"

GROUNDTRUTH_FILE = "groundTruth.csv"
PCD_IDX_COLUMN = 'step'


# STATIC_PATH = os.path.join(BASE_PATH, STATIC_FOLDER)
DYNAMIC_PATH = os.path.join(BASE_PATH, DYNAMIC_FOLDER)

In [4]:
NONPLAYER_AGENT_STR = "non_player_agents"
dynamic_non_player_type_list = ["vehicle", "pedestrian"]

In [5]:
def parse_clean_list(clean_str_list):
    location_str = "location"
    orientation_str = "orientation"
    rotation_str = "rotation"
    bounding_box_str = "bounding_box"
    extent_str = "extent"

    x_str = "x"
    y_str = "y"
    z_str = "z"
    yaw_str = "yaw"

    non_player_dict = {}
    non_player_dict["id"] = clean_str_list[1]
    non_player_dict["type"] = clean_str_list[2]

    # Extract all list indices
    location_idx = clean_str_list.index(location_str)
    orientation_idx = clean_str_list.index(orientation_str)
    rotation_idx = clean_str_list.index(rotation_str)
    bounding_box_idx = clean_str_list.index(bounding_box_str)
    extent_idx = clean_str_list.index(extent_str)

    # Get relevant list subsets
    location_str_list = clean_str_list[location_idx:orientation_idx]
    rotation_str_list = clean_str_list[rotation_idx:bounding_box_idx]
    extent_str_list = clean_str_list[extent_idx:]

    # Extract location: x,y,z
    non_player_dict["location_x"] = float(location_str_list[location_str_list.index(x_str)+1])
    non_player_dict["location_y"] = float(location_str_list[location_str_list.index(y_str)+1])
    non_player_dict["location_z"] = float(location_str_list[location_str_list.index(z_str)+1])

    # Extract heading rotation: yaw
    non_player_dict["yaw"] = float(rotation_str_list[rotation_str_list.index(yaw_str)+1])

    # Extract bounding box extent: x,y,z
    non_player_dict["extent_x"] = float(extent_str_list[extent_str_list.index(x_str)+1])
    non_player_dict["extent_y"] = float(extent_str_list[extent_str_list.index(y_str)+1])
    non_player_dict["extent_z"] = float(extent_str_list[extent_str_list.index(z_str)+1])
    
    # Get min, max extent in xy plane
    min_extent = min(non_player_dict["extent_x"], non_player_dict["extent_y"])
    max_extent = max(non_player_dict["extent_x"], non_player_dict["extent_y"])
    non_player_dict["min_half_extent"] = min_extent / 2
    non_player_dict["max_half_extent"] = max_extent / 2
    
    return non_player_dict

def get_relative_yaw(theta1, theta2):
    rel_yaw = theta2 - theta1
    if rel_yaw > 180:
        actual_rel_yaw = rel_yaw - 360
    elif rel_yaw <= -180:
        actual_rel_yaw = rel_yaw + 360
    else:
        actual_rel_yaw = rel_yaw
    return actual_rel_yaw

def pose2matrix(translation_list, rotation_angle_list):
    trans_vec = np.array(translation_list)
    rot_ang = [np.deg2rad(ang) for ang in rotation_angle_list ]
    rot_mat = transforms3d.euler.euler2mat(rot_ang[0], rot_ang[1], rot_ang[2])
    zoom = np.ones(3)
    transform_mat = transforms3d.affines.compose(trans_vec, rot_mat, zoom)
    return transform_mat

In [6]:
sub_folder = "17"
# for sub_folder in os.listdir(DYNAMIC_PATH):

In [7]:
GROUNDTRUTH_PATH = os.path.join(DYNAMIC_PATH, sub_folder, GROUNDTRUTH_FILE)
if os.path.exists(GROUNDTRUTH_PATH):
    df_gt = pd.read_csv(GROUNDTRUTH_PATH)
else:
    print("Error: Could not find ground truth csv at path : {}".format(GROUNDTRUTH_PATH))
#     continue

NONPLAYERS_PATH = os.path.join(DYNAMIC_PATH, sub_folder, NONPLAYERS_FOLDER)
if os.path.exists(NONPLAYERS_PATH):
    json_file_idx_list = sorted([int(file[:-5]) for file in os.listdir(NONPLAYERS_PATH)])
else:
    print("Error: Could not find non players folder at path : {}".format(NONPLAYERS_PATH))
#     continue
df_gt=df_gt[df_gt[PCD_IDX_COLUMN].isin(json_file_idx_list)]

PCD_PATH = os.path.join(DYNAMIC_PATH, sub_folder, PCD_FOLDER)
if os.path.exists(PCD_PATH):
    pcd_file_idx_list = sorted([int(file[:-4]) for file in os.listdir(PCD_PATH)])
else:
    print("Error: Could not find pcd folder at path : {}".format(PCD_PATH))
#     continue
df_gt=df_gt[df_gt[PCD_IDX_COLUMN].isin(pcd_file_idx_list)]

SEGMENT_PATH = os.path.join(DYNAMIC_PATH, sub_folder, SEGMENT_FOLDER)
if not os.path.exists(SEGMENT_PATH):
    os.makedirs(SEGMENT_PATH)
else:
    shutil.rmtree(SEGMENT_PATH)
    os.makedirs(SEGMENT_PATH)
    
STATIC_ONLY_PATH = os.path.join(DYNAMIC_PATH, sub_folder, STATIC_ONLY_FOLDER)
if not os.path.exists(STATIC_ONLY_PATH):
    os.makedirs(STATIC_ONLY_PATH)
else:
    shutil.rmtree(STATIC_ONLY_PATH)
    os.makedirs(STATIC_ONLY_PATH)

In [8]:
for gt_idx, gt_row in tqdm_notebook(df_gt.iterrows(), total=df_gt.shape[0]):
    # Read faking json file
    json_file_name = str(int(gt_row[PCD_IDX_COLUMN])) + ".json"
    json_file_path = os.path.join(NONPLAYERS_PATH, json_file_name)

    with open(json_file_path, "r") as json_file_data:
        json_file_str = json_file_data.read()

    # Parse the faking json file
    non_player_str_list = re.split(NONPLAYER_AGENT_STR, json_file_str)
    non_player_str_list

    dynamic_str_list = [non_player_str for non_player_str in non_player_str_list[1:]\
                                        for non_player_type in dynamic_non_player_type_list\
                                          if non_player_type in non_player_str]

    dataframe_list = []
    for dynamic_str in dynamic_str_list:
        clean_str_list = [some_str for some_str in re.split("[:{}\n ]", dynamic_str) if some_str]
        try:
            dataframe_list.append(parse_clean_list(clean_str_list))
        except Exception as e:
            pass
#             print(e)
#             print(clean_str_list)

    df_nonplayer = pd.DataFrame(dataframe_list)

    # Search for surrounding dynamic non players
    lidar_center = np.array([gt_row['location_x'], gt_row['location_y']])
    df_feasible_nonplayer = df_nonplayer[np.array([np.linalg.norm(lidar_center -\
                                                                  np.array([row['location_x'], row['location_y']])) \
                                                       < (LIDAR_RANGE + row['min_half_extent']) \
                                                             for idx, row in df_nonplayer.iterrows()])]


    # Read the pcd of interest
    pcd_file_name = str(int(gt_row[PCD_IDX_COLUMN])) + ".ply"
    pcd_file_path = os.path.join(DYNAMIC_PATH, sub_folder, PCD_FOLDER, pcd_file_name)
    this_pcd = o3d.read_point_cloud(pcd_file_path)

    # Align pcd to global axes
    this_pcd_align = copy.deepcopy(this_pcd)
    this_pcd_align_arr = np.asarray(this_pcd_align.points)
    this_pcd_align_arr = np.array([np.array([-y,x,-z]) for x,y,z in this_pcd_align_arr])
    this_pcd_align.points = o3d.utility.Vector3dVector(this_pcd_align_arr)

    # Initialize dynamic flag for all points
    dynamic_flag = np.full(this_pcd_align_arr.shape[0], False)

    # For every dynamic non player
    for idx, row in df_feasible_nonplayer.iterrows():
        this_pcd_tmp = copy.deepcopy(this_pcd_align)

        # Rotate pcd to global axes
        transform_mat = pose2matrix([0,0,0], [0,0,gt_row['rotation_yaw']])
        this_pcd_tmp.transform(transform_mat)

        # Transform pcd to vehicle coords
        rel_x = row['location_x'] - gt_row['location_x']
        rel_y = row['location_y'] - gt_row['location_y']
        transform_mat = pose2matrix([-rel_x,-rel_y,0], [0,0,0])
        this_pcd_tmp.transform(transform_mat)

        # Rotate pcd to vehicle axes
        transform_mat = pose2matrix([0,0,0], [0,0,row['yaw']])
        this_pcd_tmp.transform(transform_mat)

        # Find points that lie in this bounding box
        rel_extent_x = row['extent_x']
        rel_extent_y = row['extent_y']
        rel_extent_z = row['extent_z']
        this_pcd_tmp_arr = np.asarray(this_pcd_tmp.points)
        this_dynamic_flag = np.array([((x <= rel_extent_x) & (-rel_extent_x <= x) & \
                                       (y <= rel_extent_y) & (-rel_extent_y <= y))\
                                         for x,y,z in this_pcd_tmp_arr])

        dynamic_flag = dynamic_flag | this_dynamic_flag

#     print(dynamic_flag.sum())
    # Set color red for dynamic points
    color_arr = np.array([np.array([int(flag),0,0]) for flag in dynamic_flag])
    this_pcd.colors = o3d.utility.Vector3dVector(color_arr)

    # Save this pcd
    out_pcd_fname = str(int(gt_row[PCD_IDX_COLUMN])) + ".pcd"
    out_pcd_path = os.path.join(SEGMENT_PATH, out_pcd_fname)
    o3d.io.write_point_cloud(out_pcd_path, this_pcd, write_ascii=True)
    
    # Save static points pcd only
    this_pcd_static = copy.deepcopy(this_pcd)
    this_pcd_static_arr = np.asarray(this_pcd_static.points)
    this_pcd_static.points = o3d.utility.Vector3dVector(this_pcd_static_arr[~dynamic_flag])
    
    out_static_pcd_fname = str(int(gt_row[PCD_IDX_COLUMN])) + ".ply"
    out_static_pcd_path = os.path.join(STATIC_ONLY_PATH, out_static_pcd_fname)
    o3d.io.write_point_cloud(out_static_pcd_path, this_pcd_static, write_ascii=True)

In [9]:
def draw_pcd(pcd, where='mat_2d'):
    if where is 'opn_nb':
        visualizer = o3d.JVisualizer()
        visualizer.add_geometry(pcd)
        visualizer.show()
    elif where is 'opn_view':
        o3d.visualization.draw_geometries([pcd], width=1280, height=800)
    elif where is 'mat_3d':
        plt.figure()
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1], pts[:,2])
        plt.grid()
        plt.show()
    elif where is 'mat_2d':
        plt.figure()
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1])
        plt.grid()
        plt.show()
        
def draw_registration_result(src_pcd, dst_pcd, x_pt, y_pt, theta):    
    src_pcd_tmp = copy.deepcopy(src_pcd)
    dst_pcd_tmp = copy.deepcopy(dst_pcd)
    
    src_pcd_tmp.paint_uniform_color([1, 0, 0])  # red source
    dst_pcd_tmp.paint_uniform_color([0, 0, 1])  # blue target
    
    transform_mat = pose2matrix([x_pt, y_pt, 0], [0,0,theta])
    dst_pcd_tmp.transform(transform_mat)
    
    visualizer = o3d.JVisualizer()
    visualizer.add_geometry(src_pcd_tmp)
    visualizer.add_geometry(dst_pcd_tmp)
    visualizer.show()

For visualization purposes only

In [15]:
sample_step_idx = df_gt.iloc[np.random.choice(df_gt.shape[0])][PCD_IDX_COLUMN]
print(sample_step_idx)
sample_pcd_fname = str(int(sample_step_idx)) + ".pcd"
sample_pcd_path = os.path.join(SEGMENT_PATH, sample_pcd_fname)
sample_pcd = o3d.read_point_cloud(sample_pcd_path)
draw_pcd(sample_pcd, where='opn_nb')

print(sample_step_idx)
sample_pcd_fname = str(int(sample_step_idx)) + ".ply"
sample_pcd_path = os.path.join(STATIC_ONLY_PATH, sample_pcd_fname)
sample_pcd = o3d.read_point_cloud(sample_pcd_path)
draw_pcd(sample_pcd, where='opn_nb')

676.0


JVisualizer with 1 geometries

676.0


JVisualizer with 1 geometries